<a href="https://colab.research.google.com/github/gcosma/DECODEclinicalTrialCalc/blob/main/HospitalisationCalculator20Feb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Hospitalisation clinical calculator - version 20 Feb 2025

Summary of overall admissions Admission rates (per 1000 persons per year) by type are shown in Figure 29. The overall rate for adults with ID was 351.6 per 1000 persons per year, compared with 246.4 per 1000 persons per year for controls. This difference was essentially due to the higher rate among emergency admissions (182.2 vs. 67.7 per 1000 persons per year), as elective rates were similar between groups." p70, Carey et al (2017)"


Two-sample Poisson Ratio Tests (Equal Sizes)
N = 162.264
lambda1 = 0.1822
lambda2 = 0.0677
sig.level = 0.05
power = 0.9
alternative = two.sided


In [21]:
import numpy as np
from scipy import stats

def calculate_hospitalization_sample_size(rate1_per_1000, rate2_per_1000, group1_name="Group 1", group2_name="Group 2", alpha=0.05, power=0.90, test_type="two.sided"):
    """
    Calculate and display sample size requirements for hospitalization rate comparison
    with user-friendly output.
    Assumes a normal approximation for Poisson rates.
    """
    # Convert rates to per person-year
    lambda1 = rate1_per_1000 / 1000
    lambda2 = rate2_per_1000 / 1000

    # Calculate sample size using normal approximation for Poisson rates
    za = stats.norm.ppf(1 - alpha / 2)  # Two-sided test
    zb = stats.norm.ppf(power)
    n = ((za + zb)**2 * (lambda1 + lambda2)) / ((lambda1 - lambda2)**2)

    # Round up to the nearest integer
    n_rounded = int(np.ceil(n))
    total_n = n_rounded * 2

    # Print user-friendly output
    print("\nHospital Admission Study Planning Tool")
    print("=" * 50)

    print("\nSTUDY OVERVIEW")
    print("-" * 50)
    print("This tool helps you plan a study comparing hospital admission rates between two groups.")
    print("It will tell you how many people you need to include in your study.")

    print("\nGROUP CHARACTERISTICS")
    print("-" * 50)
    print(f"{group1_name}:")
    print(f"- Expected admission rate: {rate1_per_1000:.1f} admissions per 1000 people per year")

    print(f"\n{group2_name}:")
    print(f"- Expected admission rate: {rate2_per_1000:.1f} admissions per 1000 people per year")

    print("\nDIFFERENCES BETWEEN GROUPS")
    print("-" * 50)
    abs_diff = abs(rate1_per_1000 - rate2_per_1000)  # Absolute difference
    rel_risk = rate1_per_1000 / rate2_per_1000 if rate2_per_1000 != 0 else float('inf')  # Relative risk
    print(f"Absolute Difference: {abs_diff:.1f} more admissions per 1000 people per year in {group1_name}")
    print(f"Relative Difference: {group1_name} has {rel_risk:.1f} times the admission rate of {group2_name}")

    print("\nSTUDY RELIABILITY")
    print("-" * 50)
    print(f"This calculation ensures your study will be:")
    print(f"- {power * 100:.0f}% likely to detect the expected difference if it exists")
    print(f"- {(1 - alpha) * 100:.0f}% confident in avoiding false positive results")

    print("\nREQUIRED SAMPLE SIZE")
    print("-" * 50)
    print(f"To achieve these goals, you need:")
    print(f"- {n_rounded:.0f} people in {group1_name}")
    print(f"- {n_rounded:.0f} people in {group2_name}")
    print(f"- {total_n:.0f} people in total")

    print("\nTECHNICAL DETAILS")
    print("-" * 50)
    print("Statistical Method: Two-sample Poisson Ratio Test (Normal Approximation)")
    print(f"Sample size (N) = {n:.3f}")
    print(f"Rates in decimal form: {lambda1:.4f} vs {lambda2:.4f}")
    print("Assumes a normal approximation for Poisson rates. Suitable for large rates (λ > 10).")

    return n_rounded

# Example usage for emergency admissions
emergency_n = calculate_hospitalization_sample_size(
    rate1_per_1000=182.2,
    rate2_per_1000=67.7,
    group1_name="ID No Intervention Group",
    group2_name="ID Intervention Group",
    alpha=0.05,
    power=0.80
)


Hospital Admission Study Planning Tool

STUDY OVERVIEW
--------------------------------------------------
This tool helps you plan a study comparing hospital admission rates between two groups.
It will tell you how many people you need to include in your study.

GROUP CHARACTERISTICS
--------------------------------------------------
ID No Intervention Group:
- Expected admission rate: 182.2 admissions per 1000 people per year

ID Intervention Group:
- Expected admission rate: 67.7 admissions per 1000 people per year

DIFFERENCES BETWEEN GROUPS
--------------------------------------------------
Absolute Difference: 114.5 more admissions per 1000 people per year in ID No Intervention Group
Relative Difference: ID No Intervention Group has 2.7 times the admission rate of ID Intervention Group

STUDY RELIABILITY
--------------------------------------------------
This calculation ensures your study will be:
- 80% likely to detect the expected difference if it exists
- 95% confident in avo